# Evaluation of Car Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import signal
from ipyleaflet import Map, GeoJSON, Rectangle, basemaps, GeoData, AntPath

In [ ]:
#df = pd.read_csv("2023-03-30-07-47-32-722.csv")
#df = pd.read_csv("2023-04-17-18-17-37-099.csv")
#df = pd.read_csv("2023-04-17-18-22-19-825.csv")
df = pd.read_csv("2023-04-19-21-28-42-526.csv")
df.tail()

In [ ]:
# calculate time relative to first measurement timestamp
df["time"] = (df["timestamp"] - df["timestamp"][0]) / 1000000000

# correct speeds of 0.0 to nan
df["speed"][df["speed"] == 0.0] = np.nan

# correct gps positions of 0.0 to nan
df["longitude"][df["longitude"] == 0.0] = np.nan
df["latitude"][df["latitude"] == 0.0] = np.nan

df.tail()

## Filter Aceleration Signals

In [ ]:
# do low pass filter
sos = signal.butter(10, 15, 'low', fs=1000, output='sos')
filtered_x = signal.sosfilt(sos, df["accel_x"])
filtered_y = signal.sosfilt(sos, df["accel_y"])
filtered_z = signal.sosfilt(sos, df["accel_z"])

In [ ]:
%matplotlib inline
# vs %matplotlib qt
# vs %matplotlib notebook
plt.figure(figsize=(30,20))
ax1 = plt.subplot(4,1,1)
plt.title("Speed")
plt.plot(df["time"][df["speed"].notna()], df["speed"][df["speed"].notna()] * 3.6,'-o')
plt.plot(df["time"][df["obd_speed"].notna()], df["obd_speed"][df["obd_speed"].notna()],'-r')
ax2 = plt.subplot(4,1,2, sharex = ax1)
plt.title("Acceleration in X")
plt.plot(df["time"], df["accel_x"],'-')
plt.plot(df["time"], filtered_x,'r-')
ax3 = plt.subplot(4,1,3, sharex = ax1)
plt.title("Acceleration in Y")
plt.plot(df["time"], df["accel_y"],'-')
plt.plot(df["time"], filtered_y,'r-')
ax4 = plt.subplot(4,1,4, sharex = ax1)
plt.title("Acceleration in Z")
plt.plot(df["time"], df["accel_z"],'-')
plt.plot(df["time"], filtered_z,'r-')
plt.show()

In [ ]:
plt.figure(figsize=(30,20))
ax1 = plt.subplot(3,1,1)
plt.title("Speed")
plt.plot(df["time"][df["speed"].notna()], df["speed"][df["speed"].notna()] * 3.6,'-o')
plt.plot(df["time"][df["obd_speed"].notna()], df["obd_speed"][df["obd_speed"].notna()],'-r')
ax2 = plt.subplot(3,1,2, sharex = ax1)
plt.title("RPM")
plt.plot(df["time"], df["obd_rpm"],'-')
ax3 = plt.subplot(3,1,3, sharex = ax1)
plt.title("Driving Pedal Value (can be used for ACC activation status)")
plt.plot(df["time"], df["obd_throttle"],'-')
plt.show()

## Create Map

In [ ]:
def create_map(df):   
    long = df["longitude"][df["longitude"].notna()].to_numpy()
    lat = df["latitude"][df["latitude"].notna()].to_numpy()

    # stack long and lat positions
    positions = np.stack((lat,long), axis=1)

    # set map center to country of interest
    m = Map(center=positions[0].tolist(), zoom=9)

    ant_path = AntPath(
        locations=positions.tolist(), 
        dash_array=[1,10],
        delay=1000,
        color='#FF0000',
        pulse_color='#0000FF'
    )
    m.add_layer(ant_path)  

    return m

In [ ]:
m = create_map(df)
m